In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import random
import seaborn as sns
import tensorflow as tf

from keras import losses, optimizers
from keras.layers import Activation, Dense

from tensorflow import keras
from tensorflow.keras import backend as K, layers, losses, metrics, optimizers
from tensorflow.keras.activations import elu, exponential, hard_sigmoid, linear,\
    relu, sigmoid, softmax, relu, tanh
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, RMSprop, SGD
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy,\
    logcosh, mean_absolute_error, mean_squared_error, poisson

from sklearn.metrics import roc_auc_score, accuracy_score
from statistics import mean


Using TensorFlow backend.


In [2]:
# Importing data
train_dataset = pd.read_csv(r'C:\Users\kzielinski003\Desktop\Lion Predictive QC\Workflow_new_SP_DR_shiny_FS_train_dataset.csv')
test_dataset = pd.read_csv(r'C:/Users/kzielinski003/Desktop/Lion Predictive QC/Workflow_new_SP_DR_shiny_FS_test_dataset.csv')

cols_to_drop = ['Label', 'Signatory_NO_errors_last_5_days', 'Signatory_NO_cases_last_5_days',
    'Signatory_NO_errors_last_month','Signatory_NO_cases_month']

train_data = train_dataset.drop(cols_to_drop, axis=1)
test_data = test_dataset.drop(cols_to_drop, axis=1)
train_id = train_dataset.pop('GRID_ID')
test_id = test_dataset.pop('GRID_ID')
train_labels = train_dataset.pop('Label')
test_labels = test_dataset.pop('Label')

In [12]:
def random_search(train_data,
                  train_labels,
                  test_data,
                  test_labels,
                  path,
                  max_iter=100,
                  max_hour=5,
                  target='Label',
                  min_auc=0.6):

    # set values for random search
    # early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
    lr = np.arange(0.005, 0.1, 0.005)
    batch_size = np.array(range(20, 110, 10))
    units = [16, 32, 64, 128, 256]
    num_hidden_layers = [1, 2, 3, 4, 5, 6]
    epochs = [100, 150, 200, 250, 300, 350]
    dropout = np.arange(0, 0.5, 0.1)
    optimizers = [Adam, SGD]  # Nadam, RMSprop,
    losses = ['binary_crossentropy']  # 'logcosh','mean_absolute_error' ,'poisson','mean_squared_error'
    activation = [relu, tanh, elu]  #,exponential
    last_activations = [sigmoid]
    
    models_list = []
    # zmiana na listę appendowaą dictami
    results_df = pd.DataFrame(columns=['lr','batch_size','model_layers','epochs','dropout','optimizer',
                                     'losses','auc_test','auc_train'])
    # build model based on parameters choosen by talos optimalization
    i = 0
    while datetime.datetime.now().time().hour != max_hour and i!= max_iter:
        curr_num_hidden_layers=num_hidden_layers[random.randrange(0, len(num_hidden_layers), 1)]
        last_activation=last_activations[random.randrange(0, len(last_activations),1)]
        loss_function=losses[random.randrange(0, len(losses),1)]
        optimizer_function=optimizers[random.randrange(0, len(optimizers),1)]
        current_lr=lr[random.randrange(0, len(lr),1)]
        current_epoch=epochs[random.randrange(0, len(epochs),1)]
        current_batch_size=batch_size[random.randrange(0, len(batch_size),1)]
        current_dropout=dropout[random.randrange(0, len(dropout),1)]
    
        random.seed()
        model = Sequential()
        model.add(Dense(units[random.randrange(0, len(units),1)], input_dim=train_data.shape[1],
                                activation=activation[random.randrange(0, len(activation),1)]))
        model.add(Dropout(current_dropout))
        for j in range(curr_num_hidden_layers):
            model.add(Dense(units[random.randrange(0, len(units),1)],
                            activation=activation[random.randrange(0, len(activation),1)]))
        model.add(Dense(1, activation=last_activation,
                        kernel_initializer='normal'))
        model.compile(loss=loss_function,
                      optimizer=optimizer_function(lr=current_lr),
                      metrics=['acc'])
        early_stop=EarlyStopping(monitor='val_loss', min_delta=0.01, patience=30, verbose=0, mode='min', restore_best_weights=True)
        history = model.fit(train_data, train_labels, epochs=current_epoch, batch_size=current_batch_size,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop]) 
        test_predictions = model.predict(test_data)
        predictions_df_test=pd.DataFrame(test_predictions)
        train_predictions = model.predict(train_data)
        predictions_df_train=pd.DataFrame(train_predictions)
        auc_test = roc_auc_score(test_labels,predictions_df_test)
        auc_train = roc_auc_score(train_labels,predictions_df_train)
        # for regression use val loss
        #auc=min(history.history['val_loss'])
        results_df.loc[len(results_df)]=[current_lr,current_batch_size,model.get_config(),current_epoch,
                                        current_dropout,str(optimizer_function),loss_function,auc_test,auc_train]
        if auc_test > min_auc : 
            models_list.append(model)
        else :
            models_list.append(np.nan)
        i=i+1
        print(i)
        if(i%25 == 0):
            results_df.to_csv(path)
    results_df.to_csv(path)
    return [results_df, models_list]

In [13]:
random_search_results_critical_LE = random_search(train_data,train_labels,test_data,test_labels,max_iter=150,max_hour=14,path=r'C:/Users/kzielinski003/Desktop/Lion Predictive QC/nn_random_search_results.csv',min_auc=0.65)

W0226 11:47:07.806414 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:13.522951 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:13.643616 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


1


W0226 11:47:13.989633 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:20.211776 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:20.385181 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


2


W0226 11:47:20.845203 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:35.292805 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 11:47:35.513960 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


...

148


W0226 12:09:17.978666 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 12:09:25.132219 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 12:09:25.299341 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


149


W0226 12:09:25.734399 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 12:09:33.352864 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
W0226 12:09:33.519790 16464 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


150


In [14]:
random_search_results_lion=random_search_results_critical_LE

In [24]:
random_search_results_lion[1][46]

In [25]:
best_model = random_search_results_lion[1][46]

In [5]:
model = Sequential()
model.add(Dense(64, input_dim= train_data.shape[1], activation=tanh))
#create a loop making a new dense layer for the amount passed to this model.
#naming the layers helps avoid tensorflow error deep in the stack trace.
model.add(Dense(16,activation=tanh))
model.add(Dropout(0.1))

#model.add(Dropout(0.4))
#model.add(Dense(32,activation=sigmoid))
model.add(Dense(32,activation=relu))
#model.add(Dense(64,activation=relu))
model.add(Dense(32,activation=relu))
model.add(Dense(1,activation=sigmoid,kernel_initializer='normal'))
#add our classification layer.


#setup our optimizer and compile
model.compile(optimizer=SGD(lr=0.005), loss='binary_crossentropy',
             metrics=['acc'])

In [26]:
best_model.summary()

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_267 (Dense)            (None, 64)                2624      
_________________________________________________________________
dropout_48 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_268 (Dense)            (None, 128)               8320      
_________________________________________________________________
dense_269 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_270 (Dense)            (None, 64)                16448     
_________________________________________________________________
dense_271 (Dense)            (None, 1)                 65        
Total params: 60,481
Trainable params: 60,481
Non-trainable params: 0
_________________________________________________

In [27]:
for layer in best_model.layers:
    try:
        print(layer.input_shape)
    except:
        print('Dropout')

(None, 40)
(None, 64)
(None, 64)
(None, 128)
(None, 256)
(None, 64)


In [28]:
best_model.save(r'C:/Users/kzielinski003/Desktop/Lion Predictive QC/final_model_DR.h5')

In [32]:
train_data.columns

Index(['ScreenedParties', 'OwnershipLayers', 'Experience_proj',
       'Experience_team', 'Hour_numeric', 'Group_cases',
       'NO_errors_last_5_days', 'NO_cases_last_5_days', 'NO_errors_last_month',
       'NO_cases_month', 'PartyType_Subsidiary', 'TLAssigned_Makowska',
       'TLAssigned_Kolodziejczyk', 'TLAssigned_Jaszewski',
       'TLAssigned_Michalik', 'TLAssigned_Armannsson', 'TLAssigned_Rybka',
       'TLAssigned_Galtie', 'TLAssigned_Skrzynecki', 'TLAssigned_Holowacz',
       'TLAssigned_Gronowski', 'CDDRiskLevel_Normal', 'CDDRiskLevel_Low',
       'CDDRiskLevel_Increased', 'ProcessingUnit_Gdansk',
       'ProcessingUnit_MidCorp', 'ProcessingUnit_DIBA',
       'ProcessingUnit_Rotterdam', 'ProcessingUnit_Not_assigned',
       'ProcessingUnit_Midcorp', 'ESR_No_ESR_needed', 'ESR_ESR_linked',
       'CRS_FALSE', 'FATCA_FALSE', 'GreenlightedDR_FALSE', 'Day_Wednesday',
       'Day_Friday', 'Day_Tuesday', 'Day_Monday', 'Population_match_FALSE'],
      dtype='object')